In [1]:
import pandas as pd
import os
from library import start
from library import clean_tea
from library import clean_exemptions

In [2]:
tea = pd.read_csv(os.path.join(start.data_path, 'tea', 'desc_long.csv'),
                  sep=",")
tea = tea[['district', 'distname', 'year', 
          'cntyname', 'distischarter', 'rating',
          'students_num', 'students_frpl_num',
          'students_black_num', 'students_hisp_num', 'students_white_num',
          'teachers_num', 'teachers_new_num', 'teacher_turnover_num', 
          'teachers_turnover_denom', 'teachers_turnover_ratio',
          'teachers_exp_ave', 'teachers_tenure_ave',
          'teachers_nodegree_num', 'teachers_badegree_num',
          'teachers_msdegree_num', 'teachers_phddegree_num',
          'r_3rd_avescore','r_3rd_numtakers',
          'm_3rd_avescore', 'm_3rd_numtakers',
          'r_4th_avescore', 'r_4th_numtakers',
          'm_4th_avescore', 'm_4th_numtakers',
          'r_5th_avescore', 'r_5th_numtakers',
          'm_5th_avescore', 'm_5th_numtakers',
          'r_6th_avescore', 'r_6th_numtakers',
          'm_6th_avescore', 'm_6th_numtakers',
          'r_7th_avescore', 'r_7th_numtakers',
          'm_7th_avescore', 'm_7th_numtakers',
          'r_8th_avescore', 'r_8th_numtakers', 
          'm_8th_avescore', 'm_8th_numtakers',
          'alg_avescore', 'alg_numtakers', 
          'bio_avescore', 'bio_numtakers',
          'eng1_avescore', 'eng1_numtakers', 
          'eng2_avescore', 'eng2_numtakers',
          'us_avescore', 'us_numtakers']]
tea.head()

,district,distname,year,cntyname,distischarter,rating,students_num,students_frpl_num,students_black_num,students_hisp_num,...,alg_avescore,alg_numtakers,bio_avescore,bio_numtakers,eng1_avescore,eng1_numtakers,eng2_avescore,eng2_numtakers,us_avescore,us_numtakers
0,1902,CAYUGA ISD,yr1112,ANDERSON,N,NaN,590,220,35,30,...,4079.0,61.0,4049.0,60.0,2049.0,62.0,NaN,NaN,3535.0,44.0
1,1903,ELKHART ISD,yr1112,ANDERSON,N,NaN,1271,706,62,100,...,4012.0,68.0,3608.0,80.0,2040.0,79.0,NaN,NaN,NaN,NaN
2,1904,FRANKSTON ISD,yr1112,ANDERSON,N,NaN,733,425,82,76,...,3733.0,34.0,3512.0,32.0,1898.0,37.0,NaN,NaN,3503.0,37.0
3,1906,NECHES ISD,yr1112,ANDERSON,N,NaN,403,163,29,40,...,3885.0,33.0,4056.0,33.0,1968.0,34.0,NaN,NaN,NaN,NaN
4,1907,PALESTINE ISD,yr1112,ANDERSON,N,NaN,3220,2470,903,1255,...,3657.0,160.0,3847.0,196.0,1858.0,196.0,NaN,NaN,3713.0,198.0


In [3]:
laws = pd.read_csv(os.path.join(start.data_path, 'plans', 'doi_final.csv'),
                   sep=",")
laws = laws.drop(['Unnamed: 0', 'level', 'type', 'link', 'p_doi', 'date_p'],
                 axis = 1)
laws = laws.rename({'district': 'distname'}, axis = 1)
laws.head()

,distname,date,reg1_251,reg1_253,reg102_1301,reg102_1303,reg102_1305,reg102_1307,reg102_1309,reg102_1311,...,reg551_083,reg551_084,reg552_023,reg552_024,reg7_056,reg75_600,reg8_210,reg921_406,reg97_1001,reg97_1055
0,Abbott ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abernathy ISD,2017.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abilene ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Academy ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adrian ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Clean variables for merge

problems with district name from scraping

In [4]:
tea = tea.pipe(clean_exemptions.resolve_unicode_problems, 'distname')
laws = laws.pipe(clean_exemptions.resolve_unicode_problems, 'distname')

scraped names in title case, but tea all caps. change scraped distname to caps

In [5]:
laws = laws.pipe(clean_exemptions.uppercase_column, 'distname')

sometimes districts named CISD othertimes ISD. Make all ISD

In [6]:
tea = clean_exemptions.replace_column_values(tea, 'distname', 'CISD', 'ISD')
laws = clean_exemptions.replace_column_values(laws, 'distname', 'CISD', 'ISD')

fix district names that don't match

In [7]:
tea = clean_exemptions.sync_district_names(tea, 'distname')
laws = clean_exemptions.sync_district_names(laws, 'distname')

create number and non-number version of every district in tea. Then we can merge with correct version

In [10]:
tea_with_parens = tea.copy()
tea_with_parens = clean_exemptions.distnum_in_paren(tea_with_parens)
tea = pd.concat([tea, tea_with_parens])

In [12]:
tea.head()

,district,distname,year,cntyname,distischarter,rating,students_num,students_frpl_num,students_black_num,students_hisp_num,...,alg_avescore,alg_numtakers,bio_avescore,bio_numtakers,eng1_avescore,eng1_numtakers,eng2_avescore,eng2_numtakers,us_avescore,us_numtakers
0,1902,CAYUGA ISD,yr1112,ANDERSON,N,NaN,590,220,35,30,...,4079.0,61.0,4049.0,60.0,2049.0,62.0,NaN,NaN,3535.0,44.0
1,1903,ELKHART ISD,yr1112,ANDERSON,N,NaN,1271,706,62,100,...,4012.0,68.0,3608.0,80.0,2040.0,79.0,NaN,NaN,NaN,NaN
2,1904,FRANKSTON ISD,yr1112,ANDERSON,N,NaN,733,425,82,76,...,3733.0,34.0,3512.0,32.0,1898.0,37.0,NaN,NaN,3503.0,37.0
3,1906,NECHES ISD,yr1112,ANDERSON,N,NaN,403,163,29,40,...,3885.0,33.0,4056.0,33.0,1968.0,34.0,NaN,NaN,NaN,NaN
4,1907,PALESTINE ISD,yr1112,ANDERSON,N,NaN,3220,2470,903,1255,...,3657.0,160.0,3847.0,196.0,1858.0,196.0,NaN,NaN,3713.0,198.0


In [17]:
laws.head()

,distname,date,reg1_251,reg1_253,reg102_1301,reg102_1303,reg102_1305,reg102_1307,reg102_1309,reg102_1311,...,reg551_083,reg551_084,reg552_023,reg552_024,reg7_056,reg75_600,reg8_210,reg921_406,reg97_1001,reg97_1055
0,ABBOTT ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABERNATHY ISD,2017.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ABILENE ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ACADEMY ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ADRIAN ISD,2018.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Merge

In [20]:
data = laws.merge(tea, left_on='distname', right_on='distname', how='left')

In [22]:
laws.distname.nunique(), tea.district.nunique(), data.distname.nunique()

(781, 1257, 781)